In [5]:
import pandas as pd
import altair as alt

# Make charts render nicely in notebooks
alt.data_transformers.disable_max_rows()

df = pd.read_csv("../data/raw/amazon_sales_dataset.csv")
df.head()

,order_id,order_date,product_id,product_category,price,discount_percent,quantity_sold,customer_region,payment_method,rating,review_count,discounted_price,total_revenue
0,1,2022-04-13,2637,Books,128.75,10,4,North America,UPI,3.5,443,115.88,463.52
1,2,2023-03-12,2300,Fashion,302.60,20,5,Asia,Credit Card,3.7,475,242.08,1210.40
2,3,2022-09-28,3670,Sports,495.80,20,2,Europe,UPI,4.4,183,396.64,793.28
3,4,2022-04-17,2522,Books,371.95,15,4,Middle East,UPI,5.0,212,316.16,1264.64
4,5,2022-03-13,1717,Beauty,201.68,0,4,Middle East,UPI,4.6,308,201.68,806.72


In [6]:
df.info()
df.head()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          50000 non-null  int64  
 1   order_date        50000 non-null  object 
 2   product_id        50000 non-null  int64  
 3   product_category  50000 non-null  object 
 4   price             50000 non-null  float64
 5   discount_percent  50000 non-null  int64  
 6   quantity_sold     50000 non-null  int64  
 7   customer_region   50000 non-null  object 
 8   payment_method    50000 non-null  object 
 9   rating            50000 non-null  float64
 10  review_count      50000 non-null  int64  
 11  discounted_price  50000 non-null  float64
 12  total_revenue     50000 non-null  float64
dtypes: float64(4), int64(5), object(4)
memory usage: 5.0+ MB


,order_id,product_id,price,discount_percent,quantity_sold,rating,review_count,discounted_price,total_revenue
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,25000.500000,2986.848740,252.507260,13.340700,2.999400,2.996316,249.329280,218.886566,657.331475
std,14433.901067,1156.374535,143.025544,9.850694,1.415401,1.154295,144.251981,127.317681,526.223968
min,1.000000,1000.000000,5.010000,0.000000,1.000000,1.000000,0.000000,3.530000,3.840000
25%,12500.750000,1983.000000,127.840000,5.000000,2.000000,2.000000,125.000000,109.680000,240.320000
50%,25000.500000,2983.000000,252.970000,10.000000,3.000000,3.000000,250.000000,215.805000,505.410000
75%,37500.250000,3989.000000,376.335000,20.000000,4.000000,4.000000,374.000000,322.702500,968.970000
max,50000.000000,4999.000000,499.990000,30.000000,5.000000,5.000000,499.000000,499.910000,2499.550000


In [9]:
df["order_date"].min(), df["order_date"].max()

('2022-01-01', '2023-12-31')

In [10]:
df["order_date"] = pd.to_datetime(df["order_date"])
df["year"] = df["order_date"].dt.year
df["month"] = df["order_date"].dt.month

In [14]:
df = df.set_index("order_date")

sales_month_cat = (
    df.groupby("product_category")
      .resample("M")["total_revenue"]
      .sum()
      .reset_index()
)

/var/folders/xl/g3gkg0bd7vb6dbrwcn3yc5wm0000gn/T/ipykernel_1694/534368745.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample("M")["total_revenue"]


In [ ]:
trend_chart = (
    alt.Chart(sales_month_cat)
      .mark_line()
      .encode(
           x=alt.X(
              "order_date:T",
              title="Month",
              axis=alt.Axis(format="%b %Y", labelAngle=-45)
          ),
          y=alt.Y(
              "total_revenue:Q",
              title="Total Sales",
              scale=alt.Scale(domain=[150000, 300000])
          ),
          color=alt.Color("product_category:N", title="Category")
      )
      .properties(
          title="Monthly Sales Trend by Category (2022–2023)",
          width=750,
          height=400
      )
)
trend_chart

alt.Chart(...)

In [18]:
sales_summary = (
    sales_month_cat
    .groupby("product_category")["total_revenue"]
    .agg(["mean", "std"])
    .reset_index()
)

sales_summary

,product_category,mean,std
0,Beauty,231276.040417,20865.335587
1,Books,228535.959583,13401.541754
2,Electronics,227941.417917,14839.445088
3,Fashion,228338.472500,17258.193973
4,Home & Kitchen,228047.189583,21196.018964
5,Sports,225301.492500,23115.537054
